<a href="https://colab.research.google.com/github/kobrue02/evaluating-llm-generated-nlu-data/blob/main/bin/notebooks/train_eval_nlu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/kobrue02/evaluating-llm-generated-nlu-data/
%cd evaluating-llm-generated-nlu-data

In [2]:
from bin.framework.nlu_model import IntentClassifier
from bin.utils.methods import *
from sklearn.neural_network import MLPClassifier

In [3]:
model = IntentClassifier(model=MLPClassifier())
datasets = [
    "zero_shot_simple_data",
    "one_shot_simple_data",
    "few_shot_simple_data",
    "chain_of_thought_simple_data",
    "persona_based_prompt_s1_data",
    "attribute_controlled_prompt_data"
]

In [ ]:
from bin.utils.methods import load_sipgate_dataset

golden_df = load_sipgate_dataset()
golden_df = golden_df.groupby('intent').sample(n=5, random_state=42)
golden_df.head()

In [ ]:
reports = {}
for fname in datasets:
    df = load_df(fname)
    model.fit(df)
    report = model.evaluate(golden_df)
    reports[fname] = report
    model.reset()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
def create_weighted_average_table(reports):
    weighted_averages = {}
    for dataset_name, report in reports.items():
        weighted_averages[dataset_name] = report['weighted avg']['f1-score']

    df = pd.DataFrame(weighted_averages, index=['Weighted Average F1-score']).transpose()
    return df
df = create_weighted_average_table(reports)
df_transposed = df.transpose()
df_transposed
df_transposed.to_csv('output/phi_weighted_average_f1_scores.csv')

In [ ]:
df_transposed

In [ ]:
model = IntentClassifier(model=MLPClassifier())
train_df, test_df = model.split_dataset(df)
model.fit(train_df)
report = model.classification_report(test_df)
print(report)